In [6]:
from math import sqrt
from numpy import concatenate
import matplotlib as mpl
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from keras.models import Sequential
from tensorflow.keras.models import load_model
from keras.layers import LSTM, Dense, Masking
from tensorflow.keras import callbacks as cb
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf
# from hydro_helpers.scoring import render_score
# from hydro_helpers.reporting import cv_report
# from hydro_helpers._utils import _nse as NSE
# from hydro_helpers._utils import _rmse as RMSE
mpl.rcParams['figure.figsize'] = (20, 10)
mpl.rcParams['axes.grid'] = False
tf.random.set_seed(13)

In [8]:
river = 'lena'
df = pd.read_csv('train_df.txt', index_col='date')
print(df.shape)
df.index = pd.DatetimeIndex(df.index)
# print(df.mean(axis=0))
df.head()

(12361, 211)


,stage_max_3019,stage_max_3021,stage_max_3024,stage_max_3027,stage_max_3028,stage_max_3029,stage_max_3030,stage_max_3031,stage_max_3032,stage_max_3035,...,prec_30372,prec_30385,prec_30393,prec_30433,prec_30471,prec_30493,prec_31004,prec_31026,prec_31102,prec_31137
date,,,,,,,,,,,,,,,,,,,,,
1985-01-01,-23.0,103.0,106.0,25.0,258.0,51,126.0,152.0,143.0,59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.2,0.0
1985-01-02,-23.0,103.0,106.0,23.0,258.0,49,122.0,150.0,141.0,58,...,0.0,0.0,0.0,0.9,0.0,0.0,0.5,0.7,0.0,0.7
1985-01-03,-23.0,102.0,106.0,21.0,255.0,47,120.0,147.0,140.0,57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.6,1.6,0.0,0.5
1985-01-04,-24.0,102.0,106.0,19.0,252.0,45,119.0,146.0,139.0,55,...,0.0,0.0,0.6,0.3,0.0,0.2,3.0,0.2,0.3,0.3
1985-01-05,-24.0,102.0,106.0,17.0,251.0,43,119.0,145.0,135.0,53,...,0.0,0.0,0.4,0.0,0.0,0.3,0.6,0.0,0.2,1.8


In [275]:
# run once
df = df.dropna(axis=0, how='all')
df = df.dropna(axis=1, how='all')
# print(df.isna().sum())
print(df.shape)
del df['stage_max_3048']
df = df.apply(lambda x: x.fillna(x.mean()),axis=0)
# print(df.isna().sum())
df = df.apply(lambda x: x.fillna(0),axis=0)
print(df.isna().sum())

(12207, 192)
stage_max_3019        0
stage_max_3021        0
stage_max_3024        0
stage_max_3027        0
stage_max_3028        0
stage_max_3029        0
stage_max_3030        0
stage_max_3031        0
stage_max_3032        0
stage_max_3035        0
stage_max_3036        0
stage_max_3037        0
stage_max_3038        0
stage_max_3041        0
stage_max_3042        0
stage_max_3045        0
stage_max_3047        0
stage_max_3050        0
stage_max_3087        0
stage_max_3106        0
stage_max_3169        0
stage_max_3180        0
stage_max_3229        0
stage_max_3230        0
stage_max_3554        0
stage_max_3555        0
ice_thickness_3019    0
ice_thickness_3021    0
ice_thickness_3024    0
ice_thickness_3027    0
                     ..
prec_24763            0
prec_24817            0
prec_24923            0
prec_24933            0
prec_24944            0
prec_24951            0
prec_24959            0
prec_24966            0
prec_24967            0
prec_30028            0
pre

In [276]:
# df.plot(subplots=True, figsize=[20,20])
# plt.show()
corr = round(df.corr(), 2)
# целевые гидропосты
# rownames = [k for k in corr.columns[corr.columns.str.contains("3019|3027|3028|3029|3030|3035|3041|3045|3050|3230|3036")].values]
rownames = [f'stage_max_{i}' for i in [3019,3027,3028,3029,3030,3035,3041,3045,3050,3230,3036]]
# позиция целевых гидропостов
# positions = [i for i, x in enumerate(corr.columns.str.contains("3019|3027|3028|3029|3030|3035|3041|3045|3050|3230|3036]")) if x]
# print(positions)
corr.to_excel('results/lena_kirensk_corr.xlsx')
corr.loc[rownames, :].transpose().style.background_gradient(cmap='coolwarm')
# corr.style.background_gradient(cmap='coolwarm')

In [2]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])
        
    return np.array(data), np.array(labels)


def make_dataset(df):
    test_size = len(df) - int(len(df) * 0.2)
    print('Train: ', test_size)
    print('Test: ', len(df) - test_size)
    dataset = df.values
    data_mean = np.nanmean(dataset[:test_size], axis=0)
    data_std = np.nanstd(dataset[:test_size], axis=0)
    print(dataset[:test_size].shape)
#     print(np.round(data_mean, 3), np.round(data_std, 2))
    dataset = (dataset-data_mean)/data_std
#     print(dataset[:,df.columns.get_loc(target_label)])
    # shaping training and test dataset to chunks
    x_train_multi, y_train_multi = multivariate_data(dataset[:, :], dataset[:, df.columns.get_loc(target_label)], len(dataset) - test_size,
                                                     None, past_history,
                                                     future_target, STEP)
    x_val_multi, y_val_multi = multivariate_data(dataset[:, :], dataset[:, df.columns.get_loc(target_label)],
                                                 0, int(len(df) * 0.2), past_history,
                                                 future_target, STEP)
    print('\n Input training data shape:')
    print(x_train_multi.shape[-2:], y_train_multi.shape[-2:])
    print('\n Input test data shape:')
    print(x_val_multi.shape[-2:], y_val_multi.shape[-2:])
#     print('\n Single step history:')
#     print(x_train_multi[0])
#     print('\n Single step target:')
#     print(y_train_multi[0])
    return x_train_multi, y_train_multi, x_val_multi, y_val_multi, data_mean, data_std


def model_build(df, target_label):
    print(target_label)
    x_train_multi, y_train_multi, x_val_multi, y_val_multi, data_mean, data_std = make_dataset(df)
    
    # create train and test data from chunks
    train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
    val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))

    # training and test data shuffling
    train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
    val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

    multi_step_model = tf.keras.models.Sequential()
    multi_step_model.add(tf.keras.layers.LSTM(50,
                                              return_sequences=False,
                                              input_shape=x_train_multi.shape[-2:]))
    multi_step_model.add(tf.keras.layers.Dense(7))

    multi_step_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, 
                                                                beta_2=0.999, epsilon=1e-07, amsgrad=False,
                                                                name='Adam'), 
                             loss='mse')

    model_callbacks = [cb.EarlyStopping(patience=10), 
                              cb.ModelCheckpoint(filepath=f"output/"+ river + "_" + target_label + "_LSTM.h5", save_best_only=True)]
    multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=10,
                                          callbacks=model_callbacks)
    loss = multi_step_history.history['loss']
    val_loss = multi_step_history.history['val_loss']
    epochs = range(len(loss)) 
    plt.figure(figsize=[10,10])
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.legend()
    plt.show()

In [3]:
# define train/test split
target_labels = [f'stage_max_{i}' for i in [3019,3027,3028,3029,3030,3035,3041,3045,3050,3230,3036]]
print(target_labels)
# lookback parameters
past_history = 60 # how many instances a model sees for training, in compliance with other models' input
future_target = 7 # how many instances form a prediction step
STEP = 1 # step size - 1 for daily data
BATCH_SIZE = 30 
BUFFER_SIZE = 1000
EVALUATION_INTERVAL = len(df.values)/BATCH_SIZE # what data model sees for training
EPOCHS = 100 # how many times a model sees the data

['stage_max_3019', 'stage_max_3027', 'stage_max_3028', 'stage_max_3029', 'stage_max_3030', 'stage_max_3035', 'stage_max_3041', 'stage_max_3045', 'stage_max_3050', 'stage_max_3230', 'stage_max_3036']


NameError: name 'df' is not defined

In [ ]:
# run model training and saving
for l in target_labels:
    model_build(df, l)

stage_max_3019
Train:  9766
Test:  2441
(9766, 191)

 Input training data shape:
(60, 191) (9699, 7)

 Input test data shape:
(60, 191) (2381, 7)
Epoch 1/100
407/406 [==============================] - 15s 38ms/step - loss: 0.2019 - val_loss: 0.1747
Epoch 2/100
407/406 [==============================] - 14s 35ms/step - loss: 0.0719 - val_loss: 0.1848
Epoch 3/100
319/406 [======================>.......] - ETA: 3s - loss: 0.0511

In [307]:
# new data
new_df = pd.read_csv('data/lena/df_input_cp3.txt', index_col='date')
print(new_df.shape)
new_df.index = pd.DatetimeIndex(new_df.index)
new_df.head()

(12361, 211)


,stage_max_3019,stage_max_3021,stage_max_3024,stage_max_3027,stage_max_3028,stage_max_3029,stage_max_3030,stage_max_3031,stage_max_3032,stage_max_3035,...,prec_30372,prec_30385,prec_30393,prec_30433,prec_30471,prec_30493,prec_31004,prec_31026,prec_31102,prec_31137
date,,,,,,,,,,,,,,,,,,,,,
1985-01-01,-23.0,103.0,106.0,25.0,258.0,51,126.0,152.0,143.0,59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.2,0.0
1985-01-02,-23.0,103.0,106.0,23.0,258.0,49,122.0,150.0,141.0,58,...,0.0,0.0,0.0,0.9,0.0,0.0,0.5,0.7,0.0,0.7
1985-01-03,-23.0,102.0,106.0,21.0,255.0,47,120.0,147.0,140.0,57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.6,1.6,0.0,0.5
1985-01-04,-24.0,102.0,106.0,19.0,252.0,45,119.0,146.0,139.0,55,...,0.0,0.0,0.6,0.3,0.0,0.2,3.0,0.2,0.3,0.3
1985-01-05,-24.0,102.0,106.0,17.0,251.0,43,119.0,145.0,135.0,53,...,0.0,0.0,0.4,0.0,0.0,0.3,0.6,0.0,0.2,1.8


In [287]:
# validate prediction
pred = pd.DataFrame(multi_step_model.predict(x_val_multi)) * data_std[df.columns.get_loc(target_label)] + data_mean[df.columns.get_loc(target_label)]

In [10]:
# load trained model for time saving
path = os.getcwd()
target_labels =  '3019'
multi_step_model = load_model(path + "\\output\\"+ river + "_" + target_labels + "_LSTM.h5")
# validate prediction
pred = pd.DataFrame(multi_step_model.predict(x_val_multi)) * data_std[df.columns.get_loc(target_label)] + data_mean[df.columns.get_loc(target_label)]
for i in range(future_target):
    pred[i] = pred[i].shift(i)
    pred.columns = [f'pred_{i}' for i in range(1, future_target + 1)]
    start_index = len(df)-test_size
    pred.index = df[past_history:start_index].index.values
    pred['fact'] = df[past_history:start_index][target_label]
    fig, ax = plt.subplots()
    subdf = pred['1990-04-21':'1990-06-15']
    subdf.plot(ax=ax)
    subdf['fact'].plot(ax=ax, lw=5)
    subdf.diff().plot()
    subdf['fact'].diff().plot(lw=5)
    plt.show()
    subdf.head(7)

OSError: SavedModel file does not exist at: C:\Users\gorba\DataSciense\datahack\output\lena_3019_LSTM.h5\{saved_model.pbtxt|saved_model.pb}

In [ ]:
# test model


In [148]:
# расчет метрики

# квадраты стандартных отклонений по гидропостам
STATION_COEFFS = {
        3019: 185.35707752426708,
        3027: 1223.8071616577856,
        3028: 1357.4062812989373,
        3029: 1520.7730161870682,
        3030: 1765.9217904996142,
        3035: 765.3703832632036,
        3041: 443.5766934006718,
        3045: 579.1353554017562,
        3050: 612.0471238561079,
        3230: 516.6669876251401
}

def rowwise_nse(row): 
    station_id = row.station_id
    station_coeff = STATION_COEFFS[station_id]
    
    actual = row['delta_stage_max_actual']
    predicted = row['delta_stage_max_predicted']
    return np.divide(np.square(np.subtract(predicted, actual)),
            station_coeff)

def score(actual_df, predicted_df):
    merged = pd.merge(
        left=predicted_df,
        right=actual_df,
        how='right',
        on=['date', 'station_id'],
        suffixes=('_predicted', '_actual'),
    )

    merged.delta_stage_max_predicted.fillna(0, inplace=True)
    merged['error'] = merged.apply(rowwise_nse, axis=1)
    merged.dropna(inplace=True)
    if len(merged) != len(actual_df):
        return 'length of predicted df does not match actual df'
    else:
        score = np.divide(merged['error'].sum(), len(merged))
        return score